# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *



In [ ]:
#| export
import torch
device='cuda' if torch.cuda.is_available() else 'cpu'
from fastai.vision.all import *

In [ ]:
#| export
def C_z1z2(z1norm,z1norm_2,z2norm,z2norm_2,bs):
    """Input: bs*d shaped inputs z, which are bs-samples from d-dim random variables.
       Return: 0.5*corr(z1norm,z1norm_2) + 0.5*corr(z2norm,z2norm_2)
       which will be d*d dimensional, as sum of cross correlation matrices.
    """
    
    C1 =  (z1norm.T @ z2norm_2) / bs
    C2 = (z1norm_2.T @ z2norm) / bs
    return C1.pow(2) + 0.5*C2.pow(2)


As an example, suppose $bs=32$ and $d=100$. Then we can verify that the output will be $d \times d$:

In [ ]:
bs,d = 32,100
z1norm=torch.rand(32,100)
z1norm_2=torch.rand(32,100)
z2norm=torch.rand(32,100)
z2norm_2=torch.rand(32,100)

test_eq(C_z1z2(z1norm,z1norm_2,z2norm,z2norm_2,bs).shape, (d,d))


In [ ]:
#| export
class Max_Corr(nn.Module):
    """A pair of feedforward nets with one hidden layer
    """
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(ps,ps) #fc1 and fc2 belong to the first net
        self.fc2 = nn.Linear(ps,ps)

        self.fc3 = nn.Linear(ps,ps)#fc3 and fc4 belong to the first net
        self.fc4 = nn.Linear(ps,ps)

        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        
    def forward(self,x,y):

        x=self.sigmoid(self.fc1(x)) #use sigmoid on the first net
        x=self.fc2(x)
       
        y=self.relu(self.fc3(y)) #use relu on the second net
        y=self.fc4(y)

        return x,y



In [ ]:
#| export
class Cdiff_Sup:
    
    def __init__(self,I,inner_steps,bs):
        
        self.I=I
        self.inner_steps=inner_steps
        self.bs=bs
        self.max_corr = Max_Corr()
        if device == 'cuda':
            self.max_corr.cuda()
        
    def inner_step(self,z1norm,z2norm):
    
        max_corr=self.max_corr
        I=self.I
        bs=self.bs
        inner_steps=self.inner_steps

        z1norm=z1norm.detach()
        z2norm=z2norm.detach()

        max_corr = self.max_corr


        optimizer = torch.optim.Adam(list(max_corr.parameters()),lr=0.001)
        for i in range(inner_steps):
            z1norm_2,z2norm_2=max_corr(z1norm,z2norm)        
            assert (z1norm_2.shape,z2norm_2.shape) == (z1norm.shape,z2norm.shape)

            cdiff_2 = C_z1z2(z1norm=z1norm,z1norm_2=z1norm_2,z2norm=z2norm,z2norm_2=z2norm_2,bs=bs)
            inner_loss=-1*(cdiff_2*(1-I)).mean()
            optimizer.zero_grad()
            inner_loss.backward()
            optimizer.step()
        
        for p in max_corr.parameters():
            p.requires_grad=False
            
        return max_corr
    
    def __call__(self,z1norm,z2norm):
        
            max_corr =  self.inner_step(z1norm,z2norm)
            z1norm_2,z2norm_2 = max_corr(z1norm,z2norm)
            cdiff_sup = C_z1z2(z1norm=z1norm,z1norm_2=z1norm_2,z2norm=z2norm,z2norm_2=z2norm_2,bs=bs)
    
            return cdiff_sup

In [ ]:
## | hide
import nbdev; nbdev.nbdev_export()